In [ ]:
!pip install openai==0.28 gradio diffusers invisible_watermark transformers accelerate safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.9 MB/s eta

In [ ]:
import openai
import gradio as gr
from diffusers import DiffusionPipeline
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import tempfile

openai.api_key = "..."

# Function to generate a story using OpenAI's GPT-3.5 model
def generate_story_text(type_story, main_char, background, language, ending):
    try:
        # Constructing a clear and structured prompt
        story_prompt = f"You are the story teller so you can create the new story following this:\n"
        story_prompt += f"Type: {type_story}\n"
        story_prompt += f"Main Character: {main_char}\n"
        story_prompt += f"Background Story: {background}\n"
        story_prompt += f"Ending Story: {ending}\n"
        story_prompt += f"Translate the story to language: {language}\n"
        story_prompt += f"Only display the content of the story you generated, not included anything in this prompt!"

        # Generate story text
        story_completion = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=story_prompt,
            temperature=1.0,
            max_tokens=3500, #max 4097
            n=1,
            stop=None,
            timeout=None,
            logprobs=None,
            echo=False,
            stream=False
        )
        # Extract generated story text
        generated_story = story_completion.choices[0].text.strip()

        return generated_story

    except Exception as e:
        return str(e)

# Function to generate a story name
def generate_story_name(type_story, main_char, background, language, ending):
    try:
        # Prompt for generating a name for the story
        name_prompt = f"Create the name for my story following the keywords:\n"
        name_prompt += f"Type: {type_story}\n"
        name_prompt += f"Main Character: {main_char}\n"
        name_prompt += f"Background Story: {background}\n"
        name_prompt += f"Ending Story: {ending}\n"
        name_prompt += f"Translate the name to language: {language}\n"
        name_prompt += f"AND ONLY DISPLAY THE NAME OF STORY GENERATED!"

        # Generate story name
        name_completion = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=name_prompt,
            temperature=0.5,
            max_tokens=50,
            n=1,
            stop=None,
            timeout=None,
            logprobs=None,
            echo=False,
            stream=False
        )
        # Extract generated story name
        story_name = name_completion.choices[0].text.strip()

        return story_name

    except Exception as e:
        return str(e)

# Function to generate an image related to the story
def generate_story_image(type_story, main_char, background, ending):
    try:
        #Generate image model
        pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
        pipe.to("cuda")

        img_prompt = f"The story about: Main Character: {main_char}\nBackground Story: {background}\nType: {type_story}\nEnding: {ending}"

        #Expand the prompt to make it more details
        tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-small")
        model = T5ForConditionalGeneration.from_pretrained("roborovski/superprompt-v1", device_map="auto")

        input_text = "Expand the following prompt to add more detail:"

        input_ids = tokenizer(input_text + img_prompt, return_tensors="pt").input_ids.to("cuda")

        outputs = model.generate(input_ids, max_new_tokens=77)

        #Generate the image following the prompt
        image = pipe(prompt=tokenizer.decode(outputs[0])).images[0]

        return image

    except Exception as e:
        return None

# Define a function to save the story text to a file
def save_story_fn(story_text):
    try:
        # Create a temporary file
        with tempfile.NamedTemporaryFile(mode="w", delete=False, suffix=".txt") as file:
            # Write the story text to the temporary file
            file.write(story_text)
            # Get the path of the temporary file
            file_path = file.name

        # Generate HTML code for a download link
        download_link = f'<a href="data:text/plain;charset=utf-8,{story_text.replace(" ", "%20")}" style="color: blue;" download="generated_story.txt">Download Story</a>'
        return download_link
    except Exception as e:
        print("Error saving story:", e)
        return None

# Modify the interface function to include the download button
def generate_story(type_story, main_char, background, language, ending):
    # Generate story text
    generated_story = generate_story_text(type_story, main_char, background, language, ending)
    save_story = save_story_fn(generated_story)
    if generated_story is None:
        yield None, None, None, None
        return

    # Generate story name
    story_name = generate_story_name(type_story, main_char, background, language, ending)

    if story_name is None:
        yield None, None, None, None
        return

    # Return story name and text first
    yield story_name, None, generated_story, save_story

    # Generate image based on story name and text
    image = generate_story_image(type_story, main_char, background, ending)

    if image is None:
        yield None, None, None, None
    else:
        # Return image
        yield story_name, image, generated_story, save_story

# Creating the Gradio interface
inputs = [
    gr.Text(label="Type of story", placeholder="e.g., Adventure"),
    gr.Text(label="Main character", placeholder="e.g., Alice"),
    gr.Text(label="Background of the story", placeholder="e.g., In a magical forest"),
    gr.Text(label="Language", placeholder="e.g., English"),
    gr.Text(label="Type of ending", placeholder="e.g., Happy ending")
]

outputs = [
    gr.Label(label="Story Name"),
    gr.Image(label="The Image"),
    gr.Label(label="The Story"),
    gr.HTML(label="Download link")
]

title = "Storyteller"
description = "Create your own story with the help of AI! Enter details about your story and get a unique narrative along with an accompanying image."
examples = [
    ["Fantasy", "Wizard", "In a mystical land", "Vietnamese", "Good triumphs over evil"],
    ["Mystery", "Detective", "In a small town", "English", "The culprit is revealed"],
    ["Sci-fi", "Explorer", "In a distant future", "Japanese", "Discovery of a new planet"]
]

interface = gr.Interface(fn=generate_story, inputs=inputs, outputs=outputs, title=title, description=description, examples=examples, theme=gr.themes.Monochrome())

if __name__ == "__main__":
    interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f6e33c55a5b55564ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
